In [2]:
### Import packages for data manipulation

import pandas as pd
import numpy as np
import re

### Import packages to create absolute file path & make code independent of operating system

from pathlib import Path
import os.path

import warnings
warnings.filterwarnings("ignore")

### Import packages for feature extraction

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import accuracy_score, classification_report
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB

In [3]:
### Read in dataset

base_path = Path("__file__").parent

full_path = (base_path / "../../data/processed/stackoverflow_preprocessed.csv").resolve()

stackoverflow = pd.read_csv(os.path.join(full_path))

In [3]:
stackoverflow = pd.read_csv("/Users/HenriekeMax/Documents/Career_Development/GitHub/FrauenLoop_NLP_Project_2020/data/processed/stackoverflow_preprocessed.csv")

In [2]:
stackoverflow.head()

NameError: name 'stackoverflow' is not defined

In [ ]:
### Count number of words in an answer

class WordCounter(BaseEstimator, TransformerMixin):

    def __init__(self, df):
        pass

    def transform(self, df):
        ### Prepare list for results
        results = []
        ### Variable name to compute number of words on
        name = df.columns
        ### Make into list
        answer_list = df[name[0]].tolist()
        ### Compute number of words for each answer
        for answer in answer_list:
            wordcount = len(re.findall(r'\w+', str(answer)))
            results.append(wordcount)
        ### Make into a pandas df
        df_new = pd.DataFrame.from_records(results)
        ### Add suffix
        df_new = df_new.add_suffix(name)
        return df_new

    def fit(self, df, y=None):
        ### Unless error returns self
        return self

In [5]:
### Define function to count 

def wordcounter(x):
    x = len(re.findall(r'\w+', x))
    return x

In [ ]:
### Define function to count number of words in an answer

def wordcounter(x):
    x = len(re.findall(r'\w+', str(x)))
    return x

In [ ]:
### Feature extraction: Get wordcount of stackoverflow answers

stackoverflow['answer_wordcount'] = stackoverflow['answer_text_clean'].apply(wordcounter)

In [ ]:
### Check of possible patterns in wordcount and answer score

stackoverflow.groupby(['score_cat', 'answer_wordcount']).size().unstack(fill_value=0)

In [ ]:
### Make score_cat column into type category and assign numeric category codes

stackoverflow['score_cat'] = stackoverflow['score_cat'].astype('category')
stackoverflow['score_cat'] = stackoverflow.score_cat.cat.codes
stackoverflow_copy = stackoverflow.copy()

### Make integer to be able to show heatmap in next step

stackoverflow_copy['score_cat_int'] = stackoverflow_copy.score_cat.astype(int)

In [ ]:
### Heatmap to check relationship of score_cat and answer_wordcount

df = pd.DataFrame(stackoverflow_copy, columns=['score_cat_int', 'answer_wordcount'])
corrMatrix = df.corr()
sns.heatmap(corrMatrix, annot=True)
plt.show()

In [ ]:
### Creating a binary feature "code" holding info on whether or not stackoverflow answer contains code
def codecheck(x):
    x = 1 if '<code>' in x else 0
    return x

In [ ]:
### Check of possible patterns in code existence and answer score

stack_new.groupby(['score_cat', 'code_binary']).size().unstack(fill_value=0)

In [ ]:
### Creating a feature "code count" holding info on how many code-snippets an answer contains

def codecount(x):
    x = x.count("<code>")
    return x

In [ ]:
### Check of possible patterns in number of code snippets and answer score

stackoverflow['code_count'] = stackoverflow['answer_text'].apply(codecount)
stackoverflow.groupby(['score_cat', 'code_count']).size().unstack(fill_value=0)

In [7]:
vectorizer = TfidfVectorizer()

X = vectorizer.fit_transform(stackoverflow['answer_text_clean'].apply(lambda x: np.str_(x)))

feature_names = vectorizer.get_feature_names()

X = X.toarray()

In [8]:
X.shape

(30000, 138096)

In [9]:
y = stackoverflow['score_cat']

In [10]:
y.shape

(30000,)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 1)

model = BernoulliNB()
model.fit(X_train, y_train)
y_predict = model.predict(X_test)

print(accuracy_score(y_test, y_predict))
print(classification_report(y_test, y_predict))
pd.crosstab(y_test, y_predict)